In [ ]:
# imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier
import seaborn as sns
from imblearn.over_sampling import SMOTE

In [ ]:
data = np.load('X-y.npz')
X = data['X']
y = data['y']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=17)

# SMOTE to balance my dataset
smote = SMOTE(random_state=42)
x_sm, y_sm = smote.fit_resample(X_train, y_train)

X_train = np.concatenate((X_train, x_sm), axis = 0)
y_train = np.concatenate((y_train, y_sm), axis = 0)

In [ ]:
# Initialize the XGBClassifier for multi-class classification
# 'multi:softprob' outputs the predicted probability of each data point belonging to each class.
from sklearn.utils.class_weight import compute_sample_weight
weights = compute_sample_weight(class_weight='balanced', y=y_train)

xgb_classifier = XGBClassifier(objective='multi:softprob', num_class=3, gamma=1)

# Fit the classifier to the training data
xgb_classifier.fit(X_train, y_train, sample_weight=weights)

# Predict the classes for the test data

y_predict = xgb_classifier.predict(X_val)

#evaluate the classifier
accuracy = accuracy_score(y_val, y_predict)
print("Accuracy: ", accuracy)
print("\nClassification Report:\n", classification_report(y_val, y_predict))

# hyperparameter tuning on xgboost using a validation set

# Assuming y_test and y_predict are your true labels and predictions
# For multi-class classification, you need to binarize the labels for ROC and Precision-Recall curves
n_classes = 3  # Number of classes
y_test_binarized = label_binarize(y_val, classes=[0, 1, 2])
y_predict_binarized = label_binarize(y_predict, classes=[0, 1, 2])

# Plotting the confusion matrix
cm = confusion_matrix(y_val, y_predict)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='g')
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()